In [12]:
!pip install -q catboost

In [13]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar
import datetime
from datetime import datetime, timedelta, date
import warnings

#settings
warnings.filterwarnings("ignore")


In [14]:
appearances_df = pd.read_csv("../raw_data/appearances.csv")
clubs_df = pd.read_csv("../raw_data/clubs.csv")
games_df = pd.read_csv("../raw_data/games.csv")
players_df = pd.read_csv("../raw_data/players.csv")


In [15]:

features=['goals_2022','goals_against_2022', 'goals_for_2022','games_2022','assists_2022','minutes_played_2022',
          'age','height_in_cm','squad_size','term_days_remaining','position_Attack','position_Defender',
          'position_Goalkeeper','position_Midfield','sub_position_Attacking Midfield','sub_position_Central Midfield',
          'sub_position_Centre-Back','sub_position_Centre-Forward','sub_position_Defensive Midfield',
          'sub_position_Left Midfield','sub_position_Left Winger','sub_position_Left-Back','sub_position_Right Midfield',
          'sub_position_Right Winger','sub_position_Right-Back','sub_position_Second Striker','foot_Left','foot_Right','foot_Both']


In [16]:
def players_df_preproc(players_df):
    #add year to game valuations
    now = datetime.now()
    players_df['date_of_birth'] = pd.to_datetime(players_df['date_of_birth'])
    players_df = players_df[players_df['date_of_birth'].isnull() == False]
    players_df['age'] = (now - players_df['date_of_birth']).apply(lambda x: x.days) / 365.25
    players_df['age'] = players_df['age'].round().astype(int) 

    # Calculate the contract remaining of each player
    players_df['contract_expiration_date'] = pd.to_datetime(players_df['contract_expiration_date'])
    players_df = players_df[players_df['contract_expiration_date'].isnull() == False]
    players_df['term_days_remaining'] = (players_df['contract_expiration_date']- now).apply(lambda x: x.days) 
    return players_df

In [17]:
def appearances_df_preproc(appearances_df):
    # add year to player appearances
    appearances_df['datetime']=pd.to_datetime(appearances_df['date'], format="%Y-%m-%d")
    appearances_df['year']=appearances_df['datetime'].dt.year
    appearances_df = appearances_df[(appearances_df.year > 2004 ) & (appearances_df.year < 2023 )]
    return appearances_df

In [18]:
def games_and_appearances_df_preproc(games_df,appearances_df):
    games_df['datetime']=pd.to_datetime(games_df['date'], format="%Y-%m-%d")
    games_df['year']=games_df['datetime'].dt.year
    games_df = games_df[(games_df.year > 2004 ) & (games_df.year < 2023 )]
    games_and_appearances_df = appearances_df.merge(games_df, on=['game_id'], how='left')
    return games_and_appearances_df

In [21]:
players_df_preproc(players_df).head(3)

,player_id,name,current_club_id,current_club_name,country_of_citizenship,country_of_birth,city_of_birth,date_of_birth,position,sub_position,...,contract_expiration_date,current_club_domestic_competition_id,first_name,last_name,player_code,image_url,last_season,url,age,term_days_remaining
6,188076,Kevin Buchan,511,Dundee Fc,Scotland,Scotland,Dundee,1994-08-07,Attack,Centre-Forward,...,2023-06-30,SC1,Kevin,Buchan,kevin-buchan,https://img.a.transfermarkt.technology/portrai...,2012,https://www.transfermarkt.co.uk/kevin-buchan/p...,29,99
9,139735,Giannis Gesios,5219,Aok Kerkyra,Greece,Greece,Edessa,1988-08-03,Attack,Centre-Forward,...,2023-06-30,GR1,Giannis,Gesios,giannis-gesios,https://img.a.transfermarkt.technology/portrai...,2012,https://www.transfermarkt.co.uk/giannis-gesios...,35,99
10,39049,Kei Kamara,1123,Norwich City,Sierra Leone,Sierra Leone,Kenema,1984-09-01,Attack,Centre-Forward,...,2023-12-31,GB1,Kei,Kamara,kei-kamara,https://img.a.transfermarkt.technology/portrai...,2012,https://www.transfermarkt.co.uk/kei-kamara/pro...,39,283


In [22]:
appearances_df_preproc(appearances_df).head(3)

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played,datetime,year
0,2483937_52453,2483937,52453,28095,28095,2014-08-08,Haris Handzic,RU1,0,0,0,0,90,2014-08-08,2014.0
1,2479929_67064,2479929,67064,28095,4128,2014-08-03,Felicio Brown Forbes,RU1,0,0,0,0,90,2014-08-03,2014.0
2,2483937_67064,2483937,67064,28095,4128,2014-08-08,Felicio Brown Forbes,RU1,0,0,0,0,90,2014-08-08,2014.0


In [24]:
games_and_appearances_df_preproc(games_df,appearances_df).head(4)

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date_x,player_name,competition_id_x,yellow_cards,red_cards,...,club_home_name,club_away_name,home_club_manager_name,away_club_manager_name,stadium,attendance,referee,url,datetime_y,year_y
0,2483937_52453,2483937,52453,28095,28095,2014-08-08,Haris Handzic,RU1,0,0,...,Amkar Perm,Fk Ufa,Slavoljub Muslin,Igor Kolyvanov,Zvezda,8300.0,Sergey Kulikov,https://www.transfermarkt.co.uk/amkar-perm_fk-...,2014-08-08,2014.0
1,2479929_67064,2479929,67064,28095,4128,2014-08-03,Felicio Brown Forbes,RU1,0,0,...,Kuban Krasnodar,Fk Ufa,Viktor Goncharenko,Igor Kolyvanov,Kuban Stadium,12228.0,Aleksandr Egorov,https://www.transfermarkt.co.uk/kuban-krasnoda...,2014-08-03,2014.0
2,2483937_67064,2483937,67064,28095,4128,2014-08-08,Felicio Brown Forbes,RU1,0,0,...,Amkar Perm,Fk Ufa,Slavoljub Muslin,Igor Kolyvanov,Zvezda,8300.0,Sergey Kulikov,https://www.transfermarkt.co.uk/amkar-perm_fk-...,2014-08-08,2014.0
3,2484582_67064,2484582,67064,28095,4128,2014-08-13,Felicio Brown Forbes,RU1,0,0,...,Fk Ufa,Dinamo Moskau,Igor Kolyvanov,Stanislav Cherchesov,Dinamo,5350.0,Sergey Lapochkin,https://www.transfermarkt.co.uk/fk-ufa_dinamo-...,2014-08-13,2014.0


In [25]:
#create a function to collate player stats
def player_stats(player_id, season, games_df, appearances_df):
    
    appearances_df = appearances_df_preproc(appearances_df)
    games_and_appearances_df = games_and_appearances_df_preproc(games_df,appearances_df)
    df = games_and_appearances_df[games_and_appearances_df['player_id'] == player_id]
    df =  df[ df['season'] == season]    
    if (df.shape[0] == 0):
        Out = [(np.nan, season,0,0,0,0,0,0,0,0,0)]
        out_df = pd.DataFrame(data = Out, columns = ['player_id','season','goals','games',
                                                     'assists','minutes_played','goals_for',
                                                     'goals_against','clean_sheet',
                                                     'yellow_cards','red_cards'])
        return out_df    
    else:       
        df["goals_for"] = df.apply(lambda row: row['home_club_goals'] if row['home_club_id'] == row['player_club_id'] 
            else row['away_club_goals'] if row['away_club_id'] == row['player_club_id'] 
            else np.nan, axis=1)
        df["goals_against"] = df.apply(lambda row: row['away_club_goals'] if row['home_club_id'] == row['player_club_id'] 
            else row['home_club_goals'] if row['away_club_id'] == row['player_club_id'] 
            else np.nan, axis=1)
        df['clean_sheet'] = df.apply(lambda row: 1 if row['goals_against'] == 0
            else 0 if row['goals_against'] > 0
            else np.nan, axis=1)
        df = df.groupby(['player_id',"season"],as_index=False).agg({'goals': 'sum', 'game_id': 'nunique', 
                                                                    'assists': 'sum', 'minutes_played' : 'sum', 'goals_for' : 'sum',
                                                                    'goals_against' : 'sum', 'clean_sheet' : 'sum','yellow_cards':'sum','red_cards':'sum'})
        out_df = df.rename(columns={'game_id': 'games'})
        return out_df


In [ ]:
games_and_appearances_df = games_and_appearances_df_preproc(games_df,appearances_df)

stats = player_stats(290774, 2022, games_and_appearances_df)

In [9]:
def preprocessing(clubs_df,players_df,games_df,appearances_df):
    
    players_df = players_df_preproc(players_df)
    appearances_df = appearances_df_preproc(appearances_df)
    games_and_appearances_df = games_and_appearances_df_preproc(games_df,appearances_df)
    
    merged_players_df=players_df.drop(['current_club_id', 'city_of_birth', 'date_of_birth','first_name', 'last_name', 'player_code', 'image_url', 'url'], axis=1)
    merged_players_df = merged_players_df.reindex(columns = merged_players_df.columns.tolist() + ['club_value','squad_size','goals','goals_2022','games_2022','assists_2022','minutes_played_2022','goals_against_2022','goals_for_2022','clean_sheet_2022'])
    
    for player_id in merged_players_df.player_id.unique():
        club_id = players_df.current_club_id[(players_df.player_id==player_id)]
        try:
            merged_players_df.club_value[(players_df.player_id==player_id)]=int(clubs_df.total_market_value[(clubs_df.club_id==int(club_id))])
        except:
            merged_players_df.club_value[(players_df.player_id==player_id)]='NaN'  
        merged_players_df.squad_size[(players_df.player_id==player_id)]=int((clubs_df.squad_size[(clubs_df.club_id==int(club_id))]))

    columns=['player_id','games_2022','minutes_played_2022','goals_2022','assists_2022','goals_against_2022','goals_for_2022','clean_sheet_2022','name','position','sub_position','last_season','foot','height_in_cm','age','country_of_citizenship','country_of_birth','current_club_name','club_value','squad_size','current_club_domestic_competition_id','agent_name','contract_expiration_date','term_days_remaining','market_value_in_eur','highest_market_value_in_eur']
    merged_players_df=merged_players_df[columns]
    
    season = 2022 

    #iterate through players
    for index in merged_players_df.index:
        id = merged_players_df.loc[index][0]
        name = merged_players_df.loc[index][1]
        stats = player_stats(id, season, games_and_appearances_df)
  
        merged_players_df.at[index,'games_{}'.format(season)]= stats['games'][0]
        merged_players_df.at[index,'goals_{}'.format(season)]= stats['goals'][0]
        merged_players_df.at[index,'assists_{}'.format(season)]= stats['assists'][0]
        merged_players_df.at[index,'minutes_played_{}'.format(season)]= stats['minutes_played'][0]
        merged_players_df.at[index,'goals_for_{}'.format(season)]= stats['goals_for'][0]
        merged_players_df.at[index,'goals_against_{}'.format(season)]= stats['goals_against'][0]
        merged_players_df.at[index,'clean_sheet_{}'.format(season)]= stats['clean_sheet'][0]
        merged_players_df.at[index,'yellow_cards_{}'.format(season)]= stats['yellow_cards'][0]
        merged_players_df.at[index,'red_cards_{}'.format(season)]= stats['red_cards'][0]


    #drop nan
    merged_players_df=merged_players_df.dropna(subset=['market_value_in_eur'])
    merged_players_df1 = merged_players_df[(merged_players_df.current_club_domestic_competition_id=='GB1')]
    
    # convert position categories to Columns for test data
    dummies=pd.get_dummies(merged_players_df1[['position']], prefix_sep='_') 
    merged_players_df1 = pd.concat([merged_players_df1, dummies], axis=1) 
    
    # convert position categories to Columns for test data
    dummies=pd.get_dummies(merged_players_df1[['sub_position']], prefix_sep='_') 
    merged_players_df1 = pd.concat([merged_players_df1, dummies], axis=1) 
    dummies=pd.get_dummies(merged_players_df1[['foot']], prefix_sep='_') 
    merged_players_df1 = pd.concat([merged_players_df1, dummies], axis=1) 

    #separate numeric columns
    merged_players_df2 = merged_players_df1[features]
    merged_players_df2['market_value_in_eur'] = merged_players_df1['market_value_in_eur']
    
    return merged_players_df2
    

In [ ]:
player

In [10]:
players_df = players_df_preproc(players_df)
appearances_df = appearances_df_preproc(appearances_df)


In [11]:

preprocessing(clubs_df,players_df,games_df,appearances_df)

KeyboardInterrupt: 

In [ ]:
# check model

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import ensemble
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

#settings
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 1200)
pd.set_option('display.width', 1200)
warnings.filterwarnings("ignore")
%matplotlib inline


X = merged_players_df2[features].fillna(-1000)
y=merged_players_df2['market_value_in_eur']
#split test and training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=21)
comparisons_df = pd.DataFrame(columns=['model','accuracy'])
#model data
gbr = GradientBoostingRegressor(random_state=0)
svr = SVR(kernel="linear",gamma="auto")
dtr = DecisionTreeRegressor(random_state=0)
rfr = RandomForestRegressor(random_state=1)
XGBR = ensemble.GradientBoostingRegressor(n_estimators=1000, learning_rate=0.04, random_state=1)
cat= CatBoostRegressor(iterations=10,learning_rate=1,depth=3)
#lgbm=LGBMRegressor()

models=[gbr,svr,dtr,XGBR,rfr,cat]
for model in models:
    model.fit(X_train, y_train)
    #assess accurancy of model
    comparisons_df.loc[len(comparisons_df)] = [model.__class__.__name__,model.score(X_test, y_test)]
#create barchart to show performances
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = comparisons_df.model
y = comparisons_df.accuracy
ax.bar(x,y)
plt.xticks(fontsize=18, rotation=90)
plt.title('Model accuracy',fontsize=28)
plt.show()
print(comparisons_df)

In [ ]:
merged_players_df2.head(5)